In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [2]:
conf = SparkConf().setAppName('Posexplode').setMaster('yarn')\
   .set('spark.sql.legacy.timeParserPolicy', 'LEGACY')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

sc = spark.sparkContext
print("app_id".ljust(40), sc.applicationId)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/20 20:41:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/10/20 20:41:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/10/20 20:42:00 WARN DomainSocketFactory: The short-circuit local reads feature cannot be used because libhadoop cannot be loaded.
24/10/20 20:42:00 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


app_id                                   application_1727681258360_0015


In [4]:
from pyspark.sql.types import ArrayType, StructType, StructField, StringType, IntegerType, FloatType, DoubleType, DateType, LongType
from pyspark.sql.functions import col, concat, date_format, to_date, split, lit

df_list = spark.sql(" select repeat(',', {num_days}) as seq".format(num_days=31))\
    .withColumn('arr', split(col('seq'), ','))\
    .selectExpr("posexplode(arr)").drop('col')\
    .withColumn('day', col('pos') + 1).drop('pos')\
    .withColumn('event_date', to_date(concat(lit('2024'), lit('10'), col('day')), 'yyyyMMdd'))\
    .withColumn('week_day', date_format(col('event_date'), 'EEEE').alias('week_day'))

df_list.show(n=31)

+---+----------+---------+
|day|event_date| week_day|
+---+----------+---------+
|  1|2024-10-01|  Tuesday|
|  2|2024-10-02|Wednesday|
|  3|2024-10-03| Thursday|
|  4|2024-10-04|   Friday|
|  5|2024-10-05| Saturday|
|  6|2024-10-06|   Sunday|
|  7|2024-10-07|   Monday|
|  8|2024-10-08|  Tuesday|
|  9|2024-10-09|Wednesday|
| 10|2024-10-10| Thursday|
| 11|2024-10-11|   Friday|
| 12|2024-10-12| Saturday|
| 13|2024-10-13|   Sunday|
| 14|2024-10-14|   Monday|
| 15|2024-10-15|  Tuesday|
| 16|2024-10-16|Wednesday|
| 17|2024-10-17| Thursday|
| 18|2024-10-18|   Friday|
| 19|2024-10-19| Saturday|
| 20|2024-10-20|   Sunday|
| 21|2024-10-21|   Monday|
| 22|2024-10-22|  Tuesday|
| 23|2024-10-23|Wednesday|
| 24|2024-10-24| Thursday|
| 25|2024-10-25|   Friday|
| 26|2024-10-26| Saturday|
| 27|2024-10-27|   Sunday|
| 28|2024-10-28|   Monday|
| 29|2024-10-29|  Tuesday|
| 30|2024-10-30|Wednesday|
| 31|2024-10-31| Thursday|
+---+----------+---------+
only showing top 31 rows



In [5]:
spark.stop()